<a href="https://colab.research.google.com/github/Overflow-EquinoX/Otopark/blob/main/Compiler%20Design%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install graphviz


In [ ]:


import graphviz  # Syntax Tree görselleştirme için
from collections import deque  # Derivation adımları için



In [1]:
import re

def tokenize(expression):
    # Geçerli tokenları belirtiyoruz: sayılar, operatörler, parantezler ve fonksiyonlar
    token_pattern = r'\d+|[+\-*/^()!]|sin|cos'
    invalid_pattern = r'[^\d+\-*/^()!sin cos]'  # Geçersiz karakterler için

    # Geçersiz karakter kontrolü
    invalid_tokens = re.findall(invalid_pattern, expression)
    if invalid_tokens:
        raise ValueError(f"Syntax Error: Invalid token '{invalid_tokens[0]}'")

    # Geçerli tokenları yakala
    tokens = re.findall(token_pattern, expression)

    # Sayıları "NUMBER" olarak işaretle
    return ["NUMBER" if re.fullmatch(r'\d+', token) else token for token in tokens]




# Lexical Analysis (Tokenizer işlevi)

İfadeyi parçalar ve sayıları "NUMBER" olarak etiketler

token_pattern:

\d+ → Sayılar için.

[+\-*/^()!] → Temel operatörler

 !.sin|cos → Trigonometric fonksiyonlar.


Token Sınıflandırması:

Sayılar → "NUMBER" olarak etiketlenir.
Operatörler → Olduğu gibi eklenir (+, *, ^, !).
Fonksiyonlar → sin ve cos gibi adlandırılır.

Hatalı Token Kontrolü:


Tanımsız bir sembol algılanırsa Syntax Error döndürülür.




In [ ]:
grammar = {
    "E": [["T", "E'"]],
    "E'": [["+", "T", "E'"], ["-", "T", "E'"], ["ε"]],
    "T": [["F", "T'"]],
    "T'": [["*", "F", "T'"], ["/", "F", "T'"], ["ε"]],
    "F": [["(", "E", ")"], ["NUMBER"], ["sin", "(", "E", ")"], ["cos", "(", "E", ")"], ["F", "!"], ["F", "^", "F"]]
}


Dilbilgisi Tanımı (Grammar):

Dilbilgisini yorumlayacak ve analiz edecek bir yapı


In [21]:

import re

def tokenize(expression):
    # Geçerli tokenları belirtiyoruz: sayılar, operatörler, parantezler ve fonksiyonlar
    token_pattern = r'\d+|[+\-*/^()!]|sin|cos'
    invalid_pattern = r'[^\d+\-*/^()!sin cos]'  # Geçersiz karakterler için

    # Geçersiz karakter kontrolü
    invalid_tokens = re.findall(invalid_pattern, expression)
    if invalid_tokens:
        raise ValueError(f"Syntax Error: Invalid token '{invalid_tokens[0]}'")

    # Geçerli tokenları yakala
    tokens = re.findall(token_pattern, expression)

    # Sayıları "NUMBER" olarak işaretle
    return ["NUMBER" if re.fullmatch(r'\d+', token) else token for token in tokens]



def shunting_yard(tokens):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3, '!': 4}
    associativity = {'+': 'L', '-': 'L', '*': 'L', '/': 'L', '^': 'R', '!': 'R'}
    output = []
    operators = []
    last_was_operator = True  # Bir önceki tokenın operatör olup olmadığını izler

    for token in tokens:
        if token == "NUMBER":
            output.append(token)
            last_was_operator = False
        elif token in precedence:  # Operatörler
            if last_was_operator:  # Eksik operand kontrolü
                raise ValueError(f"Syntax Error: Missing operand for operator '{token}'")
            while (operators and operators[-1] in precedence and
                   ((associativity[token] == 'L' and precedence[token] <= precedence[operators[-1]]) or
                    (associativity[token] == 'R' and precedence[token] < precedence[operators[-1]]))):
                output.append(operators.pop())
            operators.append(token)
            last_was_operator = True
        elif token in ["sin", "cos"]:  # Fonksiyonlar
            operators.append(token)
            last_was_operator = True
        elif token == "(":
            operators.append(token)
            last_was_operator = True
        elif token == ")":
            while operators and operators[-1] != "(":
                output.append(operators.pop())
            if not operators:
                raise ValueError("Syntax Error: Mismatched parentheses")  # Parantez hatası
            operators.pop()  # '(' parantezini kaldır
            if operators and operators[-1] in ["sin", "cos"]:
                output.append(operators.pop())
            last_was_operator = False
        elif token == "!":
            if last_was_operator:
                raise ValueError("Syntax Error: Missing operand for '!' operator")
            output.append(token)

    # Parantezlerin dengeli olup olmadığını kontrol et
    while operators:
        if operators[-1] == "(" or operators[-1] == ")":
            raise ValueError("Syntax Error: Mismatched parentheses")
        output.append(operators.pop())

    return output






# Syntax Tree Düğüm Sınıfı
# AST oluşturmak için mevcut yapıya ekleme
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

def build_syntax_tree(postfix):
    stack = []
    for token in postfix:
        if token == "NUMBER":
            stack.append(Node("NUMBER"))
        elif token in ["+", "-", "*", "/", "^"]:
            right = stack.pop()
            left = stack.pop()
            stack.append(Node(token, left, right))
        elif token in ["sin", "cos", "!"]:  # Unary operatörler
            operand = stack.pop()
            stack.append(Node(token, operand))
    return stack[0]

def build_ast(postfix):
    # AST'de gereksiz düğümleri kaldırmak için aynı yapı kullanılır
    stack = []
    for token in postfix:
        if token == "NUMBER":
            stack.append(Node("NUMBER"))
        elif token in ["+", "-", "*", "/", "^"]:
            right = stack.pop()
            left = stack.pop()
            stack.append(Node(token, left, right))
        elif token in ["sin", "cos", "!"]:  # Unary operatörler
            operand = stack.pop()
            stack.append(Node(token, operand))
    return stack[0]

def print_syntax_tree(node, level=0):
    if node is not None:
        print_syntax_tree(node.right, level + 1)
        print(" " * 4 * level + "->", node.value)
        print_syntax_tree(node.left, level + 1)

# Test Kodu
expression = "(3 + 5) * 2"
tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)

except ValueError as e:
    print("Syntax Error:", e)






Tokens: ['(', 'NUMBER', '+', 'NUMBER', ')', '*', 'NUMBER']
Postfix Notation: ['NUMBER', 'NUMBER', '+', 'NUMBER', '*']

Syntax Tree:
    -> NUMBER
-> *
        -> NUMBER
    -> +
        -> NUMBER

Abstract Syntax Tree (AST):
    -> NUMBER
-> *
        -> NUMBER
    -> +
        -> NUMBER


#Neden Shunting Yard Algoritması Kullanıyoruz?
-Operatör Önceliği ve Parantez Yönetimi:

Shunting Yard, operatörlerin öncelik sırasını otomatik olarak işler ve parantez hatalarını kolayca yakalayabilir.

-Basitlik:

Karmaşık grammar'ları ele almak zorunda kalmadan doğrudan matematiksel ifadeler üzerinde çalışır.

-LL(1) veya LR parsing'e göre uygulaması daha kolaydır.


-Hataları Yakalama Yeteneği:

Eksik parantez, yanlış operatör sırası gibi syntax hatalarını işlem sırasında tespit edebilir.


-AST veya Syntax Tree Üretimi:

Shunting Yard çıktısı olan postfix (RPN) notasyonu, Syntax Tree veya AST oluşturmak için ideal bir başlangıç noktasıdır.

In [23]:
# Test Kodu
expression = "(3 + 5) * 2"
tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)
except ValueError as e:
    print(e)

Tokens: ['(', 'NUMBER', '+', 'NUMBER', ')', '*', 'NUMBER']
Postfix Notation: ['NUMBER', 'NUMBER', '+', 'NUMBER', '*']

Syntax Tree:
    -> NUMBER
-> *
        -> NUMBER
    -> +
        -> NUMBER

Abstract Syntax Tree (AST):
    -> NUMBER
-> *
        -> NUMBER
    -> +
        -> NUMBER


In [17]:
# Test Kodu Hata: 1
expression = "(3 + 5 * 2"
tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)
except ValueError as e:
    print(e)

Tokens: ['(', 'NUMBER', '+', 'NUMBER', '*', 'NUMBER']
Syntax Error: Mismatched parentheses


# Eksik kapanan parantez

In [18]:
# Test Kodu Hata: 2
expression = expression = "3 + * 2"

tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)
except ValueError as e:
    print(e)

Tokens: ['NUMBER', '+', '*', 'NUMBER']
Syntax Error: Missing operand for operator '*'


# Eksik operand

In [25]:
# Test Kodu Hata 3:
expression = "3 + 5 @ 2"



tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)
except ValueError as e:
    print(e)

ValueError: Syntax Error: Invalid token '@'

#Tanımsız bir karakter kullanılmış

In [24]:
# Test Kodu: Advanced Expression
expression = "sin(3) + 5 * 2 ^ 3!"
tokens = tokenize(expression)
print("Tokens:", tokens)

try:
    postfix = shunting_yard(tokens)
    print("Postfix Notation:", postfix)

    # Syntax Tree oluşturma
    syntax_tree = build_syntax_tree(postfix)
    print("\nSyntax Tree:")
    print_syntax_tree(syntax_tree)

    # AST oluşturma
    ast = build_ast(postfix)
    print("\nAbstract Syntax Tree (AST):")
    print_syntax_tree(ast)
except ValueError as e:
    print("Syntax Error:", e)


Tokens: ['sin', '(', 'NUMBER', ')', '+', 'NUMBER', '*', 'NUMBER', '^', 'NUMBER', '!']
Postfix Notation: ['NUMBER', 'sin', 'NUMBER', 'NUMBER', 'NUMBER', '!', '^', '*', '+']

Syntax Tree:
            -> !
                -> NUMBER
        -> ^
            -> NUMBER
    -> *
        -> NUMBER
-> +
    -> sin
        -> NUMBER

Abstract Syntax Tree (AST):
            -> !
                -> NUMBER
        -> ^
            -> NUMBER
    -> *
        -> NUMBER
-> +
    -> sin
        -> NUMBER
